In [ ]:
### Task 1: Image Quality Enhancement & Restoration

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar100

# Load CIFAR-100 dataset
(x_train, _), (x_test, _) = cifar100.load_data()

# Normalize images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Introduce noise
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0.0, 1.0)
x_test_noisy = np.clip(x_test_noisy, 0.0, 1.0)

# Define Autoencoder model
def build_autoencoder():
    input_img = Input(shape=(32, 32, 3))
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    return Model(input_img, decoded)

# Compile and train autoencoder
autoencoder = build_autoencoder()
autoencoder.compile(optimizer=Adam(), loss='mse')
autoencoder.fit(x_train_noisy, x_train, epochs=10, batch_size=128, validation_data=(x_test_noisy, x_test))

# Evaluate
predicted = autoencoder.predict(x_test_noisy)
psnr_value = tf.image.psnr(tf.convert_to_tensor(x_test), tf.convert_to_tensor(predicted), max_val=1.0)
print("Average PSNR:", np.mean(psnr_value.numpy()))

# Display Results
n = 5
plt.figure(figsize=(10, 5))
for i in range(n):
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test_noisy[i])
    ax.set_title("Noisy")
    ax.axis("off")

    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(predicted[i])
    ax.set_title("Restored")
    ax.axis("off")
plt.show()


Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 167s 423ms/step - loss: 0.0241 - val_loss: 0.0092
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 196s 408ms/step - loss: 0.0087 - val_loss: 0.0079
Epoch 3/10
167/391 ━━━━━━━━━━━━━━━━━━━━ 1:27 390ms/step - loss: 0.0077

In [ ]:

### Task 2: Image Classification using AlexNet and VGG16

import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision

# Load CIFAR-100 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

def evaluate_model(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total}%")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# AlexNet
alexnet = models.alexnet(pretrained=True)
alexnet.classifier[6] = nn.Linear(4096, 100)
alexnet = alexnet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)
train_model(alexnet, criterion, optimizer)
evaluate_model(alexnet)

# VGG16
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6] = nn.Linear(4096, 100)
vgg16 = vgg16.to(device)
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)
train_model(vgg16, criterion, optimizer)
evaluate_model(vgg16)